# Down-scaled multi-area model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
!pip install nested_dict dicthash

Create config file.

In [ ]:
with open('config.py', 'w') as fp:
    fp.write(
'''import os
base_path = os.path.abspath(".")
data_path = os.path.abspath("simulations")
jobscript_template = "python {base_path}/run_simulation.py {label}"
submit_cmd = "bash -c"
''')

In [ ]:
from multiarea_model import MultiAreaModel

Neurons and indegrees are both scaled down to 0.5%.
Can usually be simulated on a local machine.

**Warning: This will not yield reasonable dynamical results from the
network and is only meant to demonstrate the simulation workflow.**

In [ ]:
scale_down_to = 0.005

Specify model and simulation parameters.

In [ ]:
conn_params = {'replace_non_simulated_areas': 'het_poisson_stat',
               'g': -11.,
               
               'K_stable': 'K_stable.npy',
               'fac_nu_ext_TH': 1.2,
               'fac_nu_ext_5E': 1.125,
               'fac_nu_ext_6E': 1.41666667,
               'av_indegree_V1': 3950.}
input_params = {'rate_ext': 10.}
neuron_params = {'V0_mean': -150.,
                 'V0_sd': 50.}
network_params = {'N_scaling': scale_down_to,
                  'K_scaling': scale_down_to,
                  'fullscale_rates': 'tests/fullscale_rates.json',
                  'input_params': input_params,
                  'connection_params': conn_params,
                  'neuron_params': neuron_params}

sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1,
              'recording_dict': {'record_vm': False}}

theory_params = {'dt': 0.1}

Instantiate a multi-area model, predict firing rates from theroy, and run the simulation. 

In [ ]:
M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True,
                   theory_spec=theory_params)

In [ ]:
p, r = M.theory.integrate_siegert()
print("Mean-field theory predicts an average "
      "rate of {0:.3f} spikes/s across all populations.".format(np.mean(r[:, -1])))

In [ ]:
M.simulation.simulate()

Load spike data.

In [ ]:
data = np.loadtxt(M.simulation.data_dir + '/recordings/' + M.simulation.label + "-spikes-1-0.dat", skiprows=3)

Compute instantaneous rate per neuron across all populations.

In [ ]:
tsteps, spikecount = np.unique(data[:,1], return_counts=True)
rate = spikecount / M.simulation.params['dt'] * 1e3 / np.sum(M.N_vec)

Plot instantaneous and mean rate.

In [ ]:
fig, ax = plt.subplots()
ax.plot(tsteps, rate)
ax.plot(tsteps, np.average(rate)*np.ones(len(tsteps)), label='mean')
ax.set_title('instantaneous rate across all populations')
ax.set_xlabel('time (ms)')
ax.set_ylabel('rate (spikes / s)')
ax.set_xlim(0, sim_params['t_sim'])
ax.set_ylim(0, 50)
ax.legend()